In [9]:
# vector to id
# id to word

from transformers import RobertaTokenizer  #, RobertaForMaskedLM
import torch

tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
model = RobertaForMaskedLM.from_pretrained("roberta-large")

# input_text = "Hello, how are [MASK] today?"
# inputs = tokenizer(input_text, return_tensors="pt")

# with torch.no_grad():
#     outputs = model(**inputs)
    
# predictions = outputs.logits
# predicted_index = torch.argmax(predictions[0, inputs["input_ids"][0] == tokenizer.mask_token_id]).item()
# predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

# print(f"Masked word: {predicted_token}")


In [3]:
import os
import pickle
dataset_name =  'sst2'

data_file_path = '%s/dataset/%s_dataset.pkl'%(os.getcwd(), dataset_name)
with open(data_file_path, "rb") as f:
    sst2 = pickle.load(f)

In [4]:
sst2_train = sst2['train']
sst2_train_sentence = sst2_train['sentence']
sst2_train_sentence[:10]

['hide new secretions from the parental units ',
 'contains no wit , only labored gags ',
 'that loves its characters and communicates something rather beautiful about human nature ',
 'remains utterly satisfied to remain the same throughout ',
 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ',
 "that 's far too tragic to merit such superficial treatment ",
 'demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ',
 'of saucy ',
 "a depressed fifteen-year-old 's suicidal poetry ",
 "are more deeply thought through than in most ` right-thinking ' films "]

In [5]:
import pickle
with open('cache_VQ_model/cache_for_sst2_renewal/roberta-large_sst2_2e-05_1024_42.pkl', 'rb') as f:
    vq_model = pickle.load(f).to('cuda')    

In [7]:
with open(f'{os.getcwd()}/model_file/roberta_large_embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)
# embed

In [6]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_embedding_to_token(embeddings, word_embedding):
    embeddings = embeddings.detach()
    word_embedding = word_embedding.detach()

    similarity = cosine_similarity(embeddings, word_embedding)
    return torch.argmax(torch.tensor(similarity)).item()


In [11]:
# Function to convert embedding to token

def get_embedding_to_token(embeddings, outputs):
    for i in range(len(outputs)):
        word_embedding = outputs[i].detach()
        print(sentences_data[i])
        similarity = cosine_similarity(embeddings, word_embedding)
        for i in range(similarity.shape[1]):
            print(tokenizer.decode(similarity[:,i].argmax()), end = ' ')
        print('\n')

In [149]:
# # roberta tokenizer id to continous
# embeddings = []
# for i in range(tokenizer.decoder.__len__()):
#     embeddings.append(model.roberta.embeddings.word_embeddings(torch.tensor(i)))
# embeddings = torch.stack(embeddings)

# with open(f'{os.getcwd()}/model_file/roberta_large_embeddings.pkl', 'wb') as f:
#     pickle.dump(embeddings, f)
# embeddings

torch.Size([50265, 1024])

In [10]:
# Set the datasets
embeddings = embeddings.detach()

sentences_data = sst2_train_sentence[:20]
input_token = []
outputs = []
inputs = tokenizer(sentences_data, 
                    add_special_tokens=True, 
                    padding= 'max_length',  # 'max_length'
                    max_length= 128,  # args.max_seq_length
                    truncation=True,
                    return_attention_mask=True, 
                    return_tensors='pt')

with torch.no_grad():
    for i in range(len(sentences_data)):
        temp = model.roberta.embeddings.word_embeddings(inputs['input_ids'][i])
        input_token.append(inputs['input_ids'][0])
        outputs.append(temp)
len(outputs)

20

In [12]:
get_embedding_to_token(embeddings, outputs)

hide new secretions from the parental units 
<s> hide  new  secret ions  from  the  parental  units   </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

contains no wit , only labored gags 
<s> cont ains  no  wit ,  only  lab ored  g ags   </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

In [13]:
vq_input = outputs
labels = sst2_train['label'][:len(vq_input)]

vq_input_ = [torch.tensor(i) for i in vq_input]
labels_ = [torch.tensor(i) for i in labels]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [14]:
vq_model.eval()
augmented_data = []
total_augmented_data = []
for embs, label in zip(vq_input_, labels_):
    embs = embs.to(device = 'cuda:0')
    label = label.to(device = 'cuda:0')
    decoded_sentence = []
    for j in range(embs.shape[0]):
        text = embs[j,:].view(1,-1)
        label = label.view(-1)
        decoded_text = vq_model(text, label)[0]
        decoded_sentence.append(decoded_text)
    augmented_data = torch.cat(decoded_sentence, dim=0)
    total_augmented_data.append(augmented_data)
total_augmented_data = torch.stack(total_augmented_data, dim=0)
embeddings = torch.cat(vq_input_, dim=0)
attention_mask = inputs['attention_mask']

In [232]:
total_augmented_data = total_augmented_data.to(device = 'cpu')

In [233]:
get_embedding_to_token(embeddings, total_augmented_data)

hide new secretions from the parental units 
 in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in 

contains no wit , only labored gags 
 in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in 

In [136]:
for i in range(5):
    word_embedding = total_augmented_data[i].detach()
    print(sentences_data[i])
    similarity = cosine_similarity(embeddings, word_embedding)
    for i in range(similarity.shape[1]):
        print(tokenizer.decode(similarity[:,i].argmax()), end = ' ')    
        # print(tokenizer.decode(similarity[:,i].argmax()), end = ' ')
    print('\n')

hide new secretions from the parental units 
 in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in 

contains no wit , only labored gags 
 in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in 

that loves its characters and communicates something rather beautiful about human nature 
 in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in 

remains utterly satisfied to remain the same throughout 
 in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in 

on the worst revenge-of-the-nerds clichés the filmmakers could dredge up 
 in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in  in 



In [138]:
word_embedding

tensor([[  20.9179, -282.2273,  118.6781,  ...,  126.8360,  -90.4233,
          160.1697],
        [  22.8258, -281.2074,  118.7337,  ...,  129.4585,  -90.8051,
          158.0662],
        [  23.5059, -280.9100,  119.7326,  ...,  126.0870,  -91.5119,
          158.1617],
        ...,
        [  20.2051, -280.8430,  117.8075,  ...,  124.6517,  -88.8068,
          158.9444],
        [  20.9919, -274.6414,  120.9024,  ...,  128.2322,  -89.1516,
          151.3676],
        [  20.9607, -282.2154,  118.0498,  ...,  126.3314,  -90.2275,
          159.8801]])

In [137]:
similarity

array([[ 0.00379766,  0.00368004,  0.00349493, ...,  0.00374482,
         0.0033736 ,  0.00380875],
       [-0.02129523, -0.02142335, -0.02172077, ..., -0.02140336,
        -0.02177148, -0.02126618],
       [-0.01390019, -0.01393088, -0.01402959, ..., -0.01405553,
        -0.01495539, -0.01388545],
       ...,
       [-0.00218302, -0.00220223, -0.00255153, ..., -0.00250377,
        -0.00250476, -0.00227114],
       [-0.00326503, -0.00289528, -0.00341081, ..., -0.00375614,
        -0.00336625, -0.0032506 ],
       [-0.01848722, -0.01840778, -0.01891378, ..., -0.01885967,
        -0.01933162, -0.01846707]], dtype=float32)

In [123]:
model = model.to(device = 'cuda:0')

In [173]:
vq_input__ = torch.stack(vq_input_, dim=0)
vq_input__.shape


torch.Size([2, 13, 1024])

In [170]:
vq_input_

[tensor([[-0.1406, -0.0096,  0.0391,  ...,  0.0508, -0.0059, -0.0360],
         [-0.2047, -0.0161, -0.2656,  ...,  0.0354, -0.0816,  0.0464],
         [-0.1115, -0.0641, -0.1033,  ...,  0.0108,  0.1166, -0.2107],
         ...,
         [-0.0828, -0.0007, -0.1174,  ...,  0.1086,  0.0696, -0.0356],
         [ 0.0078, -0.0156,  0.0156,  ..., -0.0156,  0.0231,  0.0156],
         [ 0.0078, -0.0156,  0.0156,  ..., -0.0156,  0.0231,  0.0156]]),
 tensor([[-0.1406, -0.0096,  0.0391,  ...,  0.0508, -0.0059, -0.0360],
         [ 0.0876, -0.2107, -0.0619,  ...,  0.0346, -0.0101, -0.2472],
         [-0.1479, -0.0861, -0.2339,  ...,  0.0241,  0.0740,  0.1552],
         ...,
         [-0.2632, -0.0624, -0.0471,  ...,  0.0992,  0.1281, -0.0728],
         [-0.0217,  0.2372,  0.1218,  ..., -0.0779, -0.1398,  0.2028],
         [-0.0828, -0.0007, -0.1174,  ...,  0.1086,  0.0696, -0.0356]])]

In [175]:
vq_input__ = vq_input__.to(device = 'cuda:0')

In [ ]:
vq_input__

In [176]:
for j in range(model.lm_head(vq_input__).shape[0]):
    for i in range(model.lm_head(vq_input__).shape[1]):
        predicted_index = torch.argmax(model.lm_head(vq_input__)[j,i,:]).item()
        print(torch.argmax(model.lm_head(vq_input__)[j,i,:]).item())
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        print(f"Predicted token: {predicted_token}")
    print()

20485
Predicted token: NB
7387
Predicted token: aki
19569
Predicted token: ALS
16523
Predicted token: FM
9736
Predicted token: cel
19363
Predicted token: Ġoutdated
26369
Predicted token: WH
11124
Predicted token: hal
1250
Predicted token: all
20485
Predicted token: NB
20485
Predicted token: NB
39368
Predicted token: ĠMirage
39368
Predicted token: ĠMirage

20485
Predicted token: NB
2028
Predicted token: ind
4123
Predicted token: ures
9036
Predicted token: ĠDA
5857
Predicted token: ĠNation
15722
Predicted token: ķ
14461
Predicted token: Ġnavigation
17019
Predicted token: ĠPrairie
3006
Predicted token: iss
6997
Predicted token: TR
1073
Predicted token: ag
20485
Predicted token: NB
20485
Predicted token: NB



In [191]:
vq_input__

tensor([[[-0.1406, -0.0096,  0.0391,  ...,  0.0508, -0.0059, -0.0360],
         [-0.2047, -0.0161, -0.2656,  ...,  0.0354, -0.0816,  0.0464],
         [-0.1115, -0.0641, -0.1033,  ...,  0.0108,  0.1166, -0.2107],
         ...,
         [-0.0828, -0.0007, -0.1174,  ...,  0.1086,  0.0696, -0.0356],
         [ 0.0078, -0.0156,  0.0156,  ..., -0.0156,  0.0231,  0.0156],
         [ 0.0078, -0.0156,  0.0156,  ..., -0.0156,  0.0231,  0.0156]],

        [[-0.1406, -0.0096,  0.0391,  ...,  0.0508, -0.0059, -0.0360],
         [ 0.0876, -0.2107, -0.0619,  ...,  0.0346, -0.0101, -0.2472],
         [-0.1479, -0.0861, -0.2339,  ...,  0.0241,  0.0740,  0.1552],
         ...,
         [-0.2632, -0.0624, -0.0471,  ...,  0.0992,  0.1281, -0.0728],
         [-0.0217,  0.2372,  0.1218,  ..., -0.0779, -0.1398,  0.2028],
         [-0.0828, -0.0007, -0.1174,  ...,  0.1086,  0.0696, -0.0356]]],
       device='cuda:0')

In [197]:
vq_input__ = vq_input__.to(device = 'cpu')
total_augmented_data = total_augmented_data.to(device = 'cpu')

In [199]:
vq_input__

tensor([[[-0.1406, -0.0096,  0.0391,  ...,  0.0508, -0.0059, -0.0360],
         [-0.2047, -0.0161, -0.2656,  ...,  0.0354, -0.0816,  0.0464],
         [-0.1115, -0.0641, -0.1033,  ...,  0.0108,  0.1166, -0.2107],
         ...,
         [-0.0828, -0.0007, -0.1174,  ...,  0.1086,  0.0696, -0.0356],
         [ 0.0078, -0.0156,  0.0156,  ..., -0.0156,  0.0231,  0.0156],
         [ 0.0078, -0.0156,  0.0156,  ..., -0.0156,  0.0231,  0.0156]],

        [[-0.1406, -0.0096,  0.0391,  ...,  0.0508, -0.0059, -0.0360],
         [ 0.0876, -0.2107, -0.0619,  ...,  0.0346, -0.0101, -0.2472],
         [-0.1479, -0.0861, -0.2339,  ...,  0.0241,  0.0740,  0.1552],
         ...,
         [-0.2632, -0.0624, -0.0471,  ...,  0.0992,  0.1281, -0.0728],
         [-0.0217,  0.2372,  0.1218,  ..., -0.0779, -0.1398,  0.2028],
         [-0.0828, -0.0007, -0.1174,  ...,  0.1086,  0.0696, -0.0356]]])

In [213]:
vq_input__[0][0].shape

torch.Size([1024])

In [205]:
total_augmented_data[0].shape

torch.Size([13, 1024])

In [215]:
for i in range(13):
    print(cosine_similarity(vq_input__[0][i].detach().numpy(), total_augmented_data[0][i].detach().numpy().T))

0.003797658
-0.02158549
-0.013862468
-0.01153744
-0.03962185
-0.037309144
-0.022653874
-0.054282565
-0.056920398
-0.003366254
-0.018467074
0.17039949
0.17039949


In [216]:
for i in range(13):
    print(cosine_similarity(vq_input__[1][i].detach().numpy(), total_augmented_data[1][i].detach().numpy().T))

0.003797658
0.01999777
-0.007316079
-0.054570563
-0.10847142
-0.010749772
-0.021701772
-0.068836235
-0.009753025
-0.0060856505
-0.025620691
-0.003366254
-0.018467074


In [190]:
total_augmented_data

tensor([[[  20.9179, -282.2273,  118.6781,  ...,  126.8360,  -90.4233,
           160.1697],
         [  22.5677, -282.0784,  120.0120,  ...,  125.6524,  -88.9789,
           159.9776],
         [  20.8950, -282.3316,  118.0566,  ...,  126.3345,  -90.3176,
           160.0466],
         ...,
         [  20.9607, -282.2154,  118.0498,  ...,  126.3314,  -90.2275,
           159.8801],
         [  21.1212, -282.0103,  117.8725,  ...,  126.2891,  -90.3391,
           159.7347],
         [  21.1212, -282.0103,  117.8725,  ...,  126.2891,  -90.3391,
           159.7347]],

        [[  20.9179, -282.2273,  118.6781,  ...,  126.8360,  -90.4233,
           160.1697],
         [  18.8131, -277.4051,  124.7166,  ...,  132.6433,  -89.6853,
           161.3276],
         [  20.9607, -282.2154,  118.0498,  ...,  126.3314,  -90.2275,
           159.8801],
         ...,
         [  20.4835, -282.2095,  118.4985,  ...,  126.5946,  -90.5147,
           160.4159],
         [  20.9919, -274.6414,  120.902

In [143]:
for j in range(model.lm_head(total_augmented_data).shape[0]):
    for i in range(model.lm_head(total_augmented_data).shape[1]):
        predicted_index = torch.argmax(model.lm_head(total_augmented_data)[j,i,:]).item()
        print(torch.argmax(model.lm_head(total_augmented_data)[j,i,:]).item())
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

        print(f"Predicted token: {predicted_token}")
    print()

37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively

37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively
37883
Predicted token: Ġobjectively



In [128]:
# Get the predicted token
model.lm_head()

predicted_index = torch.argmax(model.lm_head(total_augmented_data)[0,:,:]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

print(f"Predicted token: {predicted_token}")

Predicted token: None


In [111]:
torch.argmax(predictions[0, 6]).item()

3267

In [114]:
predictions.shape

torch.Size([1, 9, 50265])

In [44]:
tokenizer.convert_ids_to_tokens([predicted_index])

['Ġbed']

In [32]:
predictions.shape

torch.Size([1, 9, 50265])

In [42]:
predictions[0,6].shape

torch.Size([50265])

In [43]:
predictions[0,6]

tensor([43.1920, -4.2806, 49.4451,  ...,  0.2396,  2.6224, 29.8632])

In [30]:
inputs['input_ids'][0]

tensor([    0,   133,  4758,  4005,    15,     5, 50264,     4,     2])

In [25]:
outputs.logits.shape

torch.Size([1, 9, 50265])

In [18]:
outputs.logits.shape

torch.Size([1, 12, 50265])

In [21]:
inputs

{'input_ids': tensor([[    0, 31414,     6,   141,    32,   646, 32804,   530,   742,   452,
           116,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:
inputs["input_ids"][0]

tensor([    0, 31414,     6,   141,    32,   646, 32804,   530,   742,   452,
          116,     2])

In [19]:
tokenizer.mask_token_id

50264

In [14]:
tokenizer.token

'<mask>'

In [9]:
inputs['input_ids']

tensor([[    0, 31414,     6,   141,    32,   646, 32804,   530,   742,   452,
           116,     2]])

In [4]:
predictions.shape

torch.Size([1, 12, 50265])

In [6]:
# # 모델 불러오기

# from transformers import RobertaTokenizer, RobertaForSequenceClassification
# import torch

# # Load tokenizer and model
# tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
# model = RobertaForSequenceClassification.from_pretrained('roberta-large')

In [7]:
# # 모델 저장

# import pickle

# with open( 'model_file/roberta_large.pkl', 'wb') as f:
#     pickle.dump(model, f)
# with open( 'model_file/roberta_large_tokenizer.pkl', 'wb') as f:
#     pickle.dump(tokenizer, f)

In [39]:
# from datasets import load_dataset
# import pickle

# text_dataset = {}

# for data in ['sst2', 'cola', 'mrpc', 'qqp', 'stsb', 'mnli', 'qnli', 'rte', 'wnli', 'ax']:
#     dataset = load_dataset("glue", data)
#     text_dataset[data] = dataset
    
    
#     with open('dataset/%s_dataset.pkl'%data, 'wb') as f:
#         pickle.dump(dataset, f)


In [1]:
import numpy as np
import os
os.getcwd()

'/workspace/VQ_Augmentation'

In [2]:
model_file_path = os.getcwd() + '/model_file'


In [3]:
import pickle
with open(model_file_path + '/roberta-large_class_2.pkl', "rb") as f:
    model = pickle.load(f).to(device = 'cuda:1')
with open(model_file_path + '/roberta-large_tokenizer.pkl', "rb") as f:
    tokenizer = pickle.load(f)
    

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
import os
import pickle
dataset_names =  ['sst2', 'cola', 'mrpc', 'qqp', 'stsb', 'mnli', 'qnli', 'rte', 'wnli', 'ax']
dataset = {}
for dataset_name in dataset_names:
    data_file_path = '%s/dataset/%s_dataset.pkl'%(os.getcwd(), dataset_name)
    with open(data_file_path, "rb") as f:
        temp_data = pickle.load(f)
        dataset[dataset_name] = temp_data

In [5]:
mrpc = dataset['mrpc']
qqp = dataset['qqp']

In [9]:
qqp['test']['question1']

['Would the idea of Trump and Putin in bed together scare you, given the geopolitical implications?',
 'What are the top ten Consumer-to-Consumer E-commerce online?',
 "Why don't people simply 'Google' instead of asking questions on Quora?",
 'Is it safe to invest in social trade biz?',
 'If the universe is expanding then does matter also expand?',
 'What is the plural of hypothesis?',
 'What is the application form you need for launching a company?',
 'What is Big Theta? When should I use Big Theta as opposed to big O?',
 'What are the health implications of accidentally eating a small quantity of aluminium foil?',
 'Where is the best whirlpool service center Hyderabad?',
 'Kollywood:Why do actor Vijay fans(not all ,but many) hate actor Surya?',
 'How can you secure your computer privacy?',
 'What are the best projects for beginners in Android app development?',
 'What is a security?',
 'Which Star Wars lightsaber is your favorite?',
 'What happens when aluminium reacts with concentra

In [24]:
from torch.utils.data import Dataset
import torch
from random import randint

def sum_text(texts1, texts2, self):
    full_text = np.array(texts1,dtype=object) + np.array([self.tokenizer.sep_token for _ in range(len(texts1))])+np.array(texts2,dtype=object)

    return list(full_text)


class _Make_pair(Dataset):
    def __init__(self, args, encodings, labels):
        self.args = args
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, index):
        # get the encoded input data for the given index
        if self.args.data_loader_type == 'few-shot':
            encoding = {key: tensor[index] for key, tensor in self.encodings.items()}

        # get the corresponding label for the given index
        label = torch.tensor(self.labels[index])

        return encoding, label
    
    def __len__(self):
        return len(self.labels)

In [50]:
dataset[dataset_names[5]]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [18]:
train_eval = 'train'
dataset = mrpc[train_eval]
dataset.num_rows
num_classes = len(set(dataset['label']))
num_classes
num_shots = 16

2

In [28]:
dataset[dataset.num_rows-1]

{'sentence1': "The 30-year bond US30YT = RR rose 22 / 32 for a yield of 4.31 percent , versus 4.35 percent at Wednesday 's close .",
 'sentence2': 'The 30-year bond US30YT = RR grew 1-3 / 32 for a yield of 4.30 percent , down from 4.35 percent late Wednesday .',
 'label': 0,
 'idx': 4075}

In [32]:
dataset[value]

{'sentence1': '" Any decision on Charleroi will have huge implications for regional airports in France , " he said .',
 'sentence2': '" A bad decision on Charleroi would have huge implications for state-owned regional airports in France .',
 'label': 1,
 'idx': 1857}

In [33]:
classes = set(dataset['label'])
# dataset[train_eval].num_rows

break_point = 0
tensor_index = {}
for class_ in classes:
    tensor_index[class_] = []
while True:
    value = randint(0, dataset.num_rows - 1)
    for class_ in classes:
        if dataset[value]['label'] == class_:
            if value not in tensor_index[class_]:
                if len(tensor_index[class_]) < num_shots:
                    tensor_index[class_].append(value)
                    break_point += 1
                else:
                    pass
    if break_point == num_classes * num_shots:
        break
    else:
        pass

tensor_index_total = []
for i in classes:
    tensor_index_total += tensor_index[i]


In [35]:
len(tensor_index_total)

32

In [22]:
for i in set(dataset['label']):
    print(i)

0
1


In [17]:
dataset.num_rows

3668

In [13]:
len(set( mrpc[train_eval]['label']))

2

In [12]:
aa = np.array(mrpc['train']['sentence1'],dtype=object) + np.array([tokenizer.sep_token for _ in range(len(mrpc['train']['sentence1']))])+np.array(mrpc['train']['sentence2'],dtype=object)


linetext = list(aa)

In [16]:
type(linetext)

list

In [17]:
tokenizer(linetext, 
        add_special_tokens=True, 
        padding= 'max_length',  # 'max_length'
        max_length= 128,  # args.max_seq_length
        truncation=True,
        return_attention_mask=True, 
        return_tensors='pt') 

{'input_ids': tensor([[    0, 10127,  1001,  ...,     1,     1,     1],
        [    0,   975, 26802,  ...,     1,     1,     1],
        [    0,  1213,    56,  ...,     1,     1,     1],
        ...,
        [    0,   113,   166,  ...,     1,     1,     1],
        [    0,   133, 12059,  ...,     1,     1,     1],
        [    0,   133,   389,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [19]:
mrpc['train'][0]['sentence1']

'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'

In [20]:
mrpc['train'][0]['sentence2']

'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'

In [27]:
a = ['1','2','3']
b = ['4','5','6']

[x[0] + x[1] for x in zip(a, b)]

['14', '25', '36']

In [17]:
from operator import add
map(add , mrpc['train']['sentence1'], mrpc['train']['sentence2'])

In [9]:
tokenizer.sep_token_id

2

In [11]:
tokenizer.encode(full_sentence, add_special_tokens=False,)

[0,
 10127,
 1001,
 6182,
 1238,
 39,
 2138,
 2156,
 2661,
 37,
 373,
 22,
 5,
 4562,
 22,
 2156,
 9,
 12507,
 7018,
 23817,
 39,
 1283,
 479,
 2,
 48310,
 4506,
 7,
 123,
 25,
 129,
 22,
 5,
 4562,
 22,
 2156,
 1918,
 1001,
 6182,
 1238,
 39,
 2138,
 9,
 12507,
 7018,
 23817,
 39,
 1283,
 479,
 2]

In [39]:
dataset_names

['sst2', 'cola', 'mrpc', 'qqp', 'stsb', 'mnli', 'qnli', 'rte', 'wnli', 'ax']

In [62]:
input_ids = [tokenizer.cls_token_id]
attention_mask = [1]
token_type_dis = [0]
for sent_id, input_text in enumerate()


'</s>'

In [97]:
# import logging
# import pandas as pd

# logger = logging.getLogger(__name__)


# def input_example_to_tuple(example): 
#     if example['sentence2'] is None:
#         return [example[sentence1]]
#     else:
#         return [example[sentence1], example['sentence2']]



In [95]:
mrpc['train']['sentence2'] is None

False

In [90]:
input_example_to_tuple(mrpc['train'])

KeyError: 'Column Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence . not in the dataset. Current columns in the dataset: [\'sentence1\', \'sentence2\', \'label\', \'idx\']'

In [73]:
list(zip(mrpc['train']['sentence1'], [tokenizer.sep_token for _ in range(len(mrpc['train']['sentence1']))], mrpc['train']['sentence2']))

[('Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  '</s>',
  'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'),
 ("Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
  '</s>',
  "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 ."),
 ('They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
  '</s>',
  "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale ."),
 ('Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
  '</s>',
  'Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .'),
 ('The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on th

In [ ]:
*cls**sent_0**mask*,*+sentl_1**sep+*

In [12]:
tokenizer.sep_token

'</s>'

In [10]:
tokenizer.all_special_ids

[0, 2, 3, 1, 50264]

In [9]:
encoding = tokenizer(sst2['train'][0]['sentence'], 
                        add_special_tokens=True, 
                        padding= 'max_length',  # 'max_length'
                        max_length= 128,  # args.max_seq_length
                        truncation=True,
                        return_attention_mask=True, 
                        return_tensors='pt')  # 왜인지 encoded_plus를 쓰면 버그뜸...
# args.max_seq_length = encoding['input_ids'].shape[1]
print(encoding)

{'input_ids': tensor([[    0, 37265,    92,  3556,  2485,    31,     5, 20536,  2833,  1437,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,  

In [23]:
# for i in range(8,10):
i = 9
print(i)
print(dataset_names[i])
print(dataset[dataset_names[i]])

9
ax
DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 1104
    })
})


In [38]:
print(dataset_names[2])
print(dataset[dataset_names[2]])

mrpc
DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [36]:
num = 2
for i in range(100):
    print(dataset[dataset_names[num]]['train'][i]['sentence1'],dataset[dataset_names[num]]['train'][i]['sentence2'])
    print(dataset[dataset_names[num]]['train'][i]['label'])

Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence . Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
1
Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion . Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .
0
They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added . On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .
1
Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 . Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .
0
The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange . PG & E Corp. shares jumped $ 1.63 or 8 percent to $

In [35]:
num = 2
for i in range(100):
    print(dataset[dataset_names[num]]['train'][i]['sentence2'],dataset[dataset_names[num]]['train'][i]['label'] )

Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence . 1
Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 . 0
On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale . 1
Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 . 0
PG & E Corp. shares jumped $ 1.63 or 8 percent to $ 21.03 on the New York Stock Exchange on Friday . 1
With the scandal hanging over Stewart 's company , revenue the first quarter of the year dropped 15 percent from the same period a year earlier . 1
The tech-laced Nasdaq Composite .IXIC rallied 30.46 points , or 2.04 percent , to 1,520.15 . 0
The DVD CCA appealed that decision to the U.S. Supreme Court . 1
Earnings were affected by a non-recurring $ 8 million tax benefit in the year-ago period . 0
Shares of Xoma fell 16 percent in early trade , while shares of Genentech

In [16]:
dataset[dataset_names[9]]

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 1104
    })
})

In [14]:
import os
os.getcwd()

'/workspace/VQ_Augmentation'

In [33]:
model_name = 'roberta_large'
dataset = 'colaaaaa'
vq_lr = 0.0001
num_codebook_vectors = 512
seed = 42

if not os.path.exists(f'{os.getcwd()}/testset_acc/{model_name}_{dataset}_acc.txt'):
    with open(f'{os.getcwd()}/testset_acc/{model_name}_{dataset}_acc.txt', 'w') as f:
        f.close()

with open(f'{os.getcwd()}/testset_acc/{model_name}_{dataset}_acc.txt', 'a') as f:
    f.write('vq_lr : %s, num_codebook_vectors : %s seed : %s'%(vq_lr, num_codebook_vectors, seed))
    f.write('\n')
    f.write('acc : %s'%(0.5))
    f.write('\n')

seed = 12

with open(f'{os.getcwd()}/testset_acc/{model_name}_{dataset}_acc.txt', 'a') as f:
    f.write('vq_lr : %s, num_codebook_vectors : %s seed : %s'%(vq_lr, num_codebook_vectors, seed))
    f.write('\n')
    f.write('acc : %s'%(0.5))
    f.write('\n')


In [28]:
model_name = 'roberta_large'
dataset = 'cola'
vq_lr = 0.0001
num_codebook_vectors = 512
seed = 12
with open(f'{os.getcwd()}/testset_acc/{model_name}_{dataset}_acc.txt', 'a') as f:
    f.write('vq_lr : %s, num_codebook_vectors : %s seed : %s'%(vq_lr, num_codebook_vectors, seed))
    f.write('\n')
    f.write('acc : %s'%(0.5))
    f.close()